## Data
Being able to easily load and process data is a crucial task that can make any data science more pleasant. In this notebook, we will cover most common types often encountered in data science tasks, and we will be using this data throughout the rest of this tutorial.

In [ ]:
import Pkg;
Pkg.add("BenchmarkTools")
Pkg.add("DataFrames")
Pkg.add("DelimitedFiles")
Pkg.add("CSV")
Pkg.add("XLSX")
using BenchmarkTools
using DataFrames
using DelimitedFiles
using CSV
using XLSX

# 🗃️ Get some data
In Julia, it's pretty easy to dowload a file from the web using the `download` function. But also, you can use your favorite command line commad to download files by easily switching from Julia via the `;` key. Let's try both.

Note: `download` depends on external tools such as curl, wget or fetch. So you must have one of these.

In [ ]:
?download

In [ ]:
P = download("https://raw.githubusercontent.com/nassarhuda/easy_data/master/programming_languages.csv",
    "programminglanguages.csv")

In [ ]:
;wget "https://github.com/nassarhuda/easy_data/blob/master/programminglanguages.csv"

# 📂 Read your data from text files.
The key question here is to load data from files such as `csv` files, `xlsx` files, or just raw text files. We will go over some Julia packages that will allow us to read such files very easily.

Let's start with the package `DelimitedFiles` which is in the standard library.

In [ ]:
;head programminglanguages.csv

In [ ]:
#=
readdlm(source, 
    delim::AbstractChar, 
    T::Type, 
    eol::AbstractChar; 
    header=false, 
    skipstart=0, 
    skipblanks=true, 
    use_mmap, 
    quotes=true, 
    dims, 
    comments=false, 
    comment_char='#')
=#
P,H = readdlm("programming_languages.csv",',';header=true);

In [ ]:
P

In [ ]:
H

In [ ]:
# To write to a text file, you can:
writedlm("programminglanguages_dlm.txt", P, '-')

In [ ]:
using DataFrames
C = CSV.read("programming_languages.csv");|

In [ ]:
@show typeof(C)
C[1:10,:]
# C.year #[!,:year]

In [ ]:
@show typeof(P)
P[1:10,:]

In [ ]:
names(C)

In [ ]:
names(C)
C.year
C.language
describe(C)

In [ ]:
@btime P,H = readdlm("programming_languages.csv",',';header=true);
@btime C = CSV.read("programming_languages.csv");

In [ ]:
# To write to a *.csv file using the CSV package
CSV.write("programminglanguages_CSV.csv",DataFrame(P))

In [ ]:
T = XLSX.readdata("data/zillow_data_download_april2020.xlsx", #file name
    "Sale_counts_city", #sheet name
    "A1:F9" #cell range
    )

In [ ]:
G = XLSX.readtable("data/zillow_data_download_april2020.xlsx","Sale_counts_city");

In [ ]:
G[1]

In [ ]:
G[1][1][1:10]

In [ ]:
G[2][1:10]

In [ ]:
D = DataFrame(G...) # equivalent to DataFrame(G[1],G[2])

In [ ]:
foods = ["apple", "cucumber", "tomato", "banana"]
calories = [105,47,22,105]
prices = [0.85,1.6,0.8,0.6,]
dataframe_calories = DataFrame(item=foods,calories=calories)
dataframe_prices = DataFrame(item=foods,price=prices)

In [ ]:
DF = innerjoin(dataframe_calories,dataframe_prices,on=:item)

In [ ]:
# we can also use the DataFrame constructor on a Matrix
DataFrame(T)

In [ ]:
# if you already have a dataframe: 
# XLSX.writetable("filename.xlsx", collect(DataFrames.eachcol(df)), DataFrames.names(df))
XLSX.writetable("writefile_using_XLSX.xlsx",G[1],G[2])

## ⬇️ Importing your data

Often, the data you want to import is not stored in plain text, and you might want to import different kinds of types. Here we will go over importing `jld`, `npz`, `rda`, and `mat` files. Hopefully, these four will capture the types from four common programming languages used in Data Science (Julia, Python, R, Matlab).

We will use a toy example here of a very small matrix. But the same syntax will hold for bigger files.

```
4×5 Array{Int64,2}:
 2  1446  1705  1795  1890
 3  2926  3121  3220  3405
 4  2910  3022  2937  3224
 5  1479  1529  1582  1761
 ```

In [ ]:
using JLD
jld_data = JLD.load("data/mytempdata.jld")
save("mywrite.jld", "A", jld_data)

In [ ]:
using NPZ
npz_data = npzread("data/mytempdata.npz")
npzwrite("mywrite.npz", npz_data)

In [ ]:
using RData
R_data = RData.load("data/mytempdata.rda")
# We'll need RCall to save here. https://github.com/JuliaData/RData.jl/issues/56
using RCall
@rput R_data
R"save(R_data, file=\"mywrite.rda\")"

In [ ]:
using MAT
Matlab_data = matread("data/mytempdata.mat")
matwrite("mywrite.mat",Matlab_data)

In [ ]:
@show typeof(jld_data)
@show typeof(npz_data)
@show typeof(R_data)
@show typeof(Matlab_data)
;

In [ ]:
Matlab_data

# 🔢 Time to process the data from Julia
We will mainly cover `Matrix` (or `Vector`), `DataFrame`s, and `dict`s (or dictionaries). Let's bring back our programming languages dataset and start playing it the matrix it's stored in.

In [ ]:
P

Here are some quick questions we might want to ask about this simple data.
- Which year was was a given language invented?
- How many languages were created in a given year?

In [ ]:
# Q1: Which year was was a given language invented?
function year_created(P,language::String)
    loc = findfirst(P[:,2] .== language)
    return P[loc,1]
end
year_created(P,"Julia")

In [ ]:
year_created(P,"W")

In [ ]:
function year_created_handle_error(P,language::String)
    loc = findfirst(P[:,2] .== language)
    !isnothing(loc) && return P[loc,1]
    error("Error: Language not found.")
end
year_created_handle_error(P,"W")

In [ ]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P,year::Int64)
    year_count = length(findall(P[:,1].==year))
    return year_count
end
how_many_per_year(P,2011)

In [ ]:
P_df = C #DataFrame(year = P[:,1], language = P[:,2]) # or DataFrame(P)

In [ ]:
# Even better, since we know the types of each column, we can create the DataFrame as follows:
# P_df = DataFrame(year = Int.(P[:,1]), language = string.(P[:,2]))

In [ ]:
# Q1: Which year was was a given language invented?
# it's a little more intuitive and you don't need to remember the column ids
function year_created(P_df,language::String)
    loc = findfirst(P_df.language .== language)
    return P_df.year[loc]
end
year_created(P_df,"Julia")

In [ ]:
year_created(P_df,"W")

In [ ]:
function year_created_handle_error(P_df,language::String)
    loc = findfirst(P_df.language .== language)
    !isnothing(loc) && return P_df.year[loc]
    error("Error: Language not found.")
end
year_created_handle_error(P_df,"W")

In [ ]:
# Q2: How many languages were created in a given year?
function how_many_per_year(P_df,year::Int64)
    year_count = length(findall(P_df.year.==year))
    return year_count
end
how_many_per_year(P_df,2011)

In [ ]:
# A quick example to show how to build a dictionary
Dict([("A", 1), ("B", 2),(1,[1,2])])

In [ ]:
P_dictionary = Dict{Integer,Vector{String}}()

In [ ]:
P_dictionary[67] = ["julia","programming"]

In [ ]:
# this is not gonna work.
P_dictionary["julia"] = 7

Now, let's populate the dictionary with years as keys and vectors that hold all the programming languages created in each year as their values. Even though this looks like more work, we often need to do it just once.

In [ ]:
dict = Dict{Integer,Vector{String}}()
for i = 1:size(P,1)
    year,lang = P[i,:]
    if year in keys(dict)
        dict[year] = push!(dict[year],lang) 
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        dict[year] = [lang]
    end
end

In [ ]:
# Though a smarter way to do this is:
curyear = P_df.year[1]
P_dictionary[curyear] = [P_df.language[1]]
for (i,nextyear) in enumerate(P_df.year[2:end])
    if nextyear == curyear
        #same key
        P_dictionary[curyear] = push!(P_dictionary[curyear],P_df.language[i+1])
        # note that push! is not our favorite thing to do in Julia, 
        # but we're focusing on correctness rather than speed here
    else
        curyear = nextyear
        P_dictionary[curyear] = [P_df.language[i+1]]
    end
end

In [ ]:
length(keys(P_dictionary))

In [ ]:
length(unique(P[:,1]))

In [ ]:
# Q1: Which year was was a given language invented?
# now instead of looking in one long vector, we will look in many small vectors
function year_created(P_dictionary,language::String)
    keys_vec = collect(keys(P_dictionary))
    lookup = map(keyid -> findfirst(P_dictionary[keyid].==language),keys_vec)
    # now the lookup vector has `nothing` or a numeric value. We want to find the index of the numeric value.
    return keys_vec[findfirst((!isnothing).(lookup))]
end
year_created(P_dictionary,"Julia")

In [ ]:
# Q2: How many languages were created in a given year?
how_many_per_year(P_dictionary,year::Int64) = length(P_dictionary[year])
how_many_per_year(P_dictionary,2011)

# 📝 A note about missing data

In [ ]:
# assume there were missing values in our dataframe
P[1,1] = missing
P_df = DataFrame(year = P[:,1], language = P[:,2])

In [ ]:
dropmissing(P_df)

# Finally...
After finishing this notebook, you should be able to:
- [ ] dowload a data file from the web given a url
- [ ] load data from a file from a text file via DelimitedFiles or CSV
- [ ] write your data to a text file or csv file
- [ ] load data from file types xlsx, jld, npz, mat, rda
- [ ] write your data to an xlsx file, jld, npz, mat, rda
- [ ] store data in a 2D array (`Matrix`), or `DataFrame` or `Dict`
- [ ] write functions to perform basic lookups on `Matrix`, `DataFrame`, and `Dict` types
- [ ] use some of the basic functions on `DataFrame`s such as: `dropmissing`, `describe`, `by`, and `join`

# 🥳 One cool finding

Julia was created in 2012

## Linear Algebra
A lot of the Data Science methods we will see in this tutorial require some understanding of linear algebra, and in this notebook we will focus on how Julia handles matrices, the types that exist, and how to call basic linear algebra tasks.

In [ ]:
# some packages we will use
import Pkg;
Pkg.add("LinearAlgebra")
Pkg.add("SparseArrays")
Pkg.add("Images")
Pkg.add("MAT")
using LinearAlgebra
using SparseArrays
using Images
using MAT

![title](data/matrix_storage.png)
### 🟢Getting started

In [ ]:
A = rand(10,10); # created a random matrix of size 10-by-10
Atranspose = A' # matrix transpose
A = A*Atranspose; # matrix multiplication

In [ ]:
@show A[11] == A[1,2];

In [ ]:
b = rand(10); #created a random vector of size 10
x = A\b; #x is the solutions to the linear system Ax=b
@show norm(A*x-b)
;

A few things that are noteworthy: 
- `A` is a `Matrix` type, and `b` is a `Vector` type.
- The transpose function creates a matrix of type `Adjoint`.
- `\` is always the recommended way to solve a linear system. You almost never want to call the `inv` function

In [ ]:
@show typeof(A)
@show typeof(b)
@show typeof(rand(1,10))
@show typeof(Atranspose)
;

In [ ]:
Matrix{Float64} == Array{Float64,2}

In [ ]:
Vector{Float64} == Array{Float64,1}

In [ ]:
Atranspose

`adjoint` in julia is a lazy adjoint -- often, we can easily perform Linear Algebra operations such as `A*A'` without actually transposing the matrix.

In [ ]:
?adjoint

In [ ]:
Atranspose.parent

In [ ]:
sizeof(A)

In [ ]:
# To actually copy the matrix
B = copy(Atranspose)

In [ ]:
sizeof(B)

In [ ]:
?\

### 🟢Factorizations
A common tool used in Linear Algebra is matrix factorizations. These factorizations are often used to solve linear systems like `Ax=b`, and as we will see later in this tutorial... `Ax=b` comes up in a lot of Data Science problems

#### LU factorization
L\*U = P\*A

In [ ]:
luA = lu(A)

In [ ]:
norm(luA.L*luA.U - luA.P*A)

#### QR factorization
Q\*R = A

In [ ]:
qrA = qr(A)

In [ ]:
norm(qrA.Q*qrA.R - A)

#### Cholesky factorization, note that A needs to be symmetric positive definite
L\*L' = A 

In [ ]:
isposdef(A)

In [ ]:
cholA = cholesky(A)

In [ ]:
norm(cholA.L*cholA.U - A)

In [ ]:
cholA.L

In [ ]:
cholA.U

In [ ]:
factorize(A)

In [ ]:
?factorize

In [ ]:
?diagm

In [ ]:
# convert(Diagonal{Int64,Array{Int64,1}},diagm([1,2,3]))
Diagonal([1,2,3])

In [ ]:
I(3)

### 🟢Sparse Linear Algebra
Sparse matrices are stored in Compressed Sparse Column (CSC) form

In [ ]:
using SparseArrays
S = sprand(5,5,2/5)

In [ ]:
S.rowval

In [ ]:
Matrix(S)

In [ ]:
S.colptr

In [ ]:
S.m

### 🟢Images as matrices
Let's get to the more "data science-y" side. We will do so by working with images (which can be viewed as matrices), and we will use the `SVD` decomposition.

First let's load an image. I chose this image as it has a lot of details.

In [ ]:
X1 = load("data/khiam-small.jpg")

In [ ]:
@show typeof(X1)
X1[1,1] # this is pixel [1,1]

In [ ]:
Xgray = Gray.(X1)

In [ ]:
R = map(i->X1[i].r,1:length(X1))
R = Float64.(reshape(R,size(X1)...))

G = map(i->X1[i].g,1:length(X1))
G = Float64.(reshape(G,size(X1)...))

B = map(i->X1[i].b,1:length(X1))
B = Float64.(reshape(B,size(X1)...))
;

In [ ]:
Z = zeros(size(R)...) 
# just a matrix of all zeros of equal size as the image
RGB.(Z,G,Z)

In [ ]:
Xgrayvalues = Float64.(Xgray)

In [ ]:
SVD_V = svd(Xgrayvalues)

In [ ]:
norm(SVD_V.U*diagm(SVD_V.S)*SVD_V.V' - Xgrayvalues)

In [ ]:
# use the top 4 singular vectors/values to form a new image
u1 = SVD_V.U[:,1]
v1 = SVD_V.V[:,1]
img1 = SVD_V.S[1]*u1*v1'

i = 2
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 += SVD_V.S[i]*u1*v1'

i = 3
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 += SVD_V.S[i]*u1*v1'

i = 4
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 += SVD_V.S[i]*u1*v1'

In [ ]:
Gray.(img1)

In [ ]:
i = 1:100
u1 = SVD_V.U[:,i]
v1 = SVD_V.V[:,i]
img1 = u1*spdiagm(0=>SVD_V.S[i])*v1'
Gray.(img1)

In [ ]:
norm(Xgrayvalues-img1)

In [ ]:
M = matread("data/face_recog_qr.mat")

In [ ]:
q = reshape(M["V2"][:,1],192,168)
Gray.(q)

In [ ]:
b = q[:]

In [ ]:
A = M["V2"][:,2:end]
x = A\b #Ax=b
Gray.(reshape(A*x,192,168))

In [ ]:
norm(A*x-b)

In [ ]:
qv = q+rand(size(q,1),size(q,2))*0.5
qv = qv./maximum(qv)
Gray.(qv)

In [ ]:
b = qv[:];

In [ ]:
x = A\b
norm(A*x-b)

In [ ]:
Gray.(reshape(A*x,192,168))

# Finally...
After finishing this notebook, you should be able to:
- [ ] reshape and vectorize a matrix
- [ ] apply basic linear algebra operations such as transpose, matrix-matrix product, and solve a linear systerm
- [ ] call a linear algebra factorization on your matrix
- [ ] use SVD to created a compressed version of an image
- [ ] solve the face recognition problem via a least square approach
- [ ] create a sparse matrix, and call the components of the Compressed Sparse Column storage
- [ ] list a few types of matrices Julia uses (diagonal, upper triangular,...)
- [ ] (unrelated to linear algebra): load an image, convert it to grayscale, and extract the RGB layers

# 🥳 One cool finding

We can solve a simple form of the face recognition problem even when a face image has been distorted with wrong pixels. Example, one of our inputs was this image: <img src="data/0201.png" width="100">

And we were able to detect this face to be closest to the input image: <img src="data/0202.png" width="100">

## Statistics
Having a solid understanding of statistics in data science allows us to understand our data better, and allows us to create a quantifiable evaluation of any future conclusions.

In [ ]:
import Pkg
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("RDatasets")
Pkg.add("Plots")
Pkg.add("StatsPlots")
Pkg.add("KernelDensity")
Pkg.add("Distributions")
Pkg.add("LinearAlgebra")
Pkg.add("HypothesisTests")
Pkg.add("PyCall")
Pkg.add("MLBase")
using Statistics
using StatsBase
using RDatasets
using Plots
using StatsPlots
using KernelDensity
using Distributions
using LinearAlgebra
using HypothesisTests
using PyCall
using MLBase

In this notebook, we will use eruption data on the faithful geyser. The data will contain wait times between every consecutive times the geyser goes off and the length of the eruptions.
<img src="data/faithful.JPG" width="400">

In [ ]:
D = dataset("datasets","faithful")
@show names(D)
D

In [ ]:
describe(D)

In [ ]:
eruptions = D[!,:Eruptions]
scatter(eruptions,label="eruptions")
waittime = D[!,:Waiting]
scatter!(waittime,label="wait time")

### 🔵Statistics plots
As you can see, this doesn't tell us much about the data... Let's try some statistical plots

In [ ]:
boxplot(["eruption length"],transpose(eruptions),legend=false,size=(200,400),whisker_width=1,ylabel="time in minutes")

Statistical plots such as a box plot (and a violin plot as we will see in notebook `12. Visualization`), can provide a much better understanding of the data. Here, we immediately see that the median time of each eruption is about 4 minutes.

The next plot we will see is a histogram plot.

In [ ]:
histogram(eruptions,label="eruptions")

In [ ]:
?histogram

In [ ]:
histogram(eruptions,bins=:sqrt,label="eruptions")

In [ ]:
p=kde(eruptions)

If we want the histogram and the kernel density graph to be aligned we need to remember that the "density contribution" of every point added to one of these histograms is `1/(nb of elements)*bin width`. Read more about kernel density estimates on its wikipedia page https://en.wikipedia.org/wiki/Kernel_density_estimation

In [ ]:
histogram(eruptions,label="eruptions")
plot!(p.x,p.density .* length(eruptions), linewidth=3,color=2,label="kde fit") # nb of elements*bin width

In [ ]:
histogram(eruptions,bins=:sqrt,label="eruptions")
plot!(p.x,p.density .* length(eruptions) .*0.2, linewidth=3,color=2,label="kde fit") # nb of elements*bin width

In [ ]:
myrandomvector = randn(100_000)
histogram(myrandomvector)
p=kde(myrandomvector)
plot!(p.x,p.density .* length(myrandomvector) .*0.1, linewidth=3,color=2,label="kde fit") # nb of elements*bin width

### 🔵Probability distributions
Another way to generate the same plot is via using the `Distributions` package and choosing the probability distribution you want, and then drawing random numbers from it. As an example, we will use `d = Normal()` below.

In [ ]:
d = Normal()
myrandomvector = rand(d,100000)
histogram(myrandomvector)
p=kde(myrandomvector)
plot!(p.x,p.density .* length(myrandomvector) .*0.1, linewidth=3,color=2,label="kde fit") # nb of elements*bin width

In [ ]:
b = Binomial(40) 
myrandomvector = rand(b,1000000)
histogram(myrandomvector)
p=kde(myrandomvector)
plot!(p.x,p.density .* length(myrandomvector) .*0.5,color=2,label="kde fit") # nb of elements*bin width

In [ ]:
x = rand(1000)
d = fit(Normal, x)
myrandomvector = rand(d,1000)
histogram(myrandomvector,nbins=20,fillalpha=0.3,label="fit")
histogram!(x,nbins=20,linecolor = :red,fillalpha=0.3,label="myvector")

In [ ]:
x = eruptions
d = fit(Normal, x)
myrandomvector = rand(d,1000)
histogram(myrandomvector,nbins=20,fillalpha=0.3)
histogram!(x,nbins=20,linecolor = :red,fillalpha=0.3)

### 🔵Hypothesis testing
Next, we will perform hypothesis testing using the `HypothesisTests.jl` package.

In [ ]:
?OneSampleTTest

In [ ]:
myrandomvector = randn(1000)
OneSampleTTest(myrandomvector)

In [ ]:
OneSampleTTest(eruptions)

A note about p-values: Currently using the pvalue of spearman and pearson correlation from Python. But you can follow the formula here to implement your own.
https://stackoverflow.com/questions/53345724/how-to-use-julia-to-compute-the-pearson-correlation-coefficient-with-p-value

In [ ]:
scipy_stats = pyimport("scipy.stats")
@show scipy_stats.spearmanr(eruptions,waittime)
@show scipy_stats.pearsonr(eruptions,waittime)

In [ ]:
scipy_stats.pearsonr(eruptions,waittime)

In [ ]:
corspearman(eruptions,waittime)

In [ ]:
cor(eruptions,waittime)

In [ ]:
scatter(eruptions,waittime,xlabel="eruption length",
    ylabel="wait time between eruptions",legend=false,grid=false,size=(400,300))

Interesting! This means that the next time you visit Yellowstone National part ot see the faithful geysser and you have to wait for too long for it to go off, you will likely get a longer eruption! 

### 🔵AUC and Confusion matrix
Finally, we will cover basic tools you will need such as AUC scores or confusion matrix. We use the `MLBase` package for that.

In [ ]:
gt = [1, 1, 1, 1, 1, 1, 1, 2]
pred = [1, 1, 2, 2, 1, 1, 1, 1]
C = confusmat(2, gt, pred)   # compute confusion matrix
C ./ sum(C, dims=2)   # normalize per class
sum(diag(C)) / length(gt)  # compute correct rate from confusion matrix
correctrate(gt, pred)
C = confusmat(2, gt, pred)   

In [ ]:
gt = [1, 1, 1, 1, 1, 1, 1, 0];
pred = [1, 1, 0, 0, 1, 1, 1, 1]
ROC = MLBase.roc(gt,pred)
recall(ROC)
precision(ROC)

# Finally...
After finishing this notebook, you should be able to:
- [ ] generate statistics plots such as box plot, histogram, and kernel densities
- [ ] generate distributions in Julia, and draw random numbers accordingly
- [ ] fit a given set of numbers to a distribution
- [ ] compute basic evaluation metrics such as AUC and confusion matrix
- [ ] run hypothesis testing
- [ ] compute correlations and p-values

# 🥳 One cool finding
<img src="data/faithful.JPG" width="300">

If you go Yellowstone national park and you find out that the old faithful geyser is taking too long to erupt, then the wait might be worth it because you are likely to experience a longer eruption (i.e. there seems to be a high correlation between wait time and eruption time).

<img src="data/0301.png" width="400">


## Dimensionality Reduction
As the name says, dimensionality reduction is the idea of reducing your feature set to a much smaller number. Dimensionality reduction is often used in visualization of datasets to try and detect samples that are similar. We will cover three dimensionality reduction techniques here: 
1. t-SNE
2. PCA
3. umap

In [ ]:
# Packages we will use throughout this notebook
import Pkg;
Pkg.add("UMAP")
Pkg.add("Makie")
Pkg.add("XLSX")
Pkg.add("VegaDatasets")
Pkg.add("DataFrames")
Pkg.add("MultivariateStats")
Pkg.add("RDatasets")
Pkg.add("StatsBase")
Pkg.add("Statistics")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("ScikitLearn")
Pkg.add("MLBase")
Pkg.add("Distances")

using UMAP
using Makie
using XLSX
using VegaDatasets
using DataFrames
using MultivariateStats
using RDatasets
using StatsBase
using Statistics
using LinearAlgebra
using Plots
using ScikitLearn
using MLBase
using Distances

In [ ]:
C = DataFrame(VegaDatasets.dataset("cars"))

In [ ]:
dropmissing!(C)
M = Matrix(C[:,2:7])
names(C)

In [ ]:
car_origin = C[:,:Origin]
carmap = labelmap(car_origin) #from MLBase
uniqueids = labelencode(carmap,car_origin)

### 1️⃣ PCA 
We will first center the data.

In [ ]:
# center and normalize the data
data = M
data = (data .- mean(data,dims = 1))./ std(data,dims=1)

In [ ]:
# each car is now a column, PCA takes features - by - samples matrix
data'

In [ ]:
p = fit(PCA,data',maxoutdim=2)

In [ ]:
P = projection(p)

In [ ]:
P'*(data[1,:]-mean(p))

In [ ]:
Yte = MultivariateStats.transform(p, data') #notice that Yte[:,1] is the same as P'*(data[1,:]-mean(p))

In [ ]:
# reconstruct testing observations (approximately)
Xr = reconstruct(p, Yte)

In [ ]:
norm(Xr-data') # this won't be zero

In [ ]:
Plots.scatter(Yte[1,:],Yte[2,:])

In [ ]:
Plots.scatter(Yte[1,car_origin.=="USA"],Yte[2,car_origin.=="USA"],color=1,label="USA")
Plots.xlabel!("pca component1")
Plots.ylabel!("pca component2")
Plots.scatter!(Yte[1,car_origin.=="Japan"],Yte[2,car_origin.=="Japan"],color=2,label="Japan")
Plots.scatter!(Yte[1,car_origin.=="Europe"],Yte[2,car_origin.=="Europe"],color=3,label="Europe")

In [ ]:
p = fit(PCA,data',maxoutdim=3)
Yte = MultivariateStats.transform(p, data')
scatter3d(Yte[1,:],Yte[2,:],Yte[3,:],color=uniqueids,legend=false)

In [ ]:
scene = Makie.scatter(Yte[1,:],Yte[2,:],Yte[3,:],color=uniqueids)

In [ ]:
display(scene)

### 2️⃣ t-SNE
The next method we will use for dimensionality reduction is t-SNE. There are multiple ways you can call t-SNE from julia. Check out this notebook: https://github.com/nassarhuda/JuliaTutorials/blob/master/TSNE/TSNE.ipynb. But we will take this opportunity to try out something new... Call a function from the Scikit learn python package. This makes use of the package `ScikitLearn`.

In [ ]:
@sk_import manifold : TSNE
tfn = TSNE(n_components=2) #,perplexity=20.0,early_exaggeration=50)
Y2 = tfn.fit_transform(data);
Plots.scatter(Y2[:,1],Y2[:,2],color=uniqueids,legend=false,size=(400,300),markersize=3)

### 3️⃣ Next, UMAP
This will be our final dimensionality reduction method and we will use the package `UMAP` for it.

In [ ]:
L = cor(data,data,dims=2)
embedding = umap(L, 2)

In [ ]:
Plots.scatter(embedding[1,:],embedding[2,:],color=uniqueids)

For UMAP, we can create distances between every pair of observations differently, if we choose to. But even with both choices, we will see that UMAP generates a very similar pattern to what we have observed with t-SNE and PCA.


In [ ]:
L = pairwise(Euclidean(), data, data,dims=1) 
embedding = umap(-L, 2)

In [ ]:
Plots.scatter(embedding[1,:],embedding[2,:],color=uniqueids)

# Finally...
After finishing this notebook, you should be able to:
- [ ] apply tsne on your data
- [ ] apply umap on your data
- [ ] apply pca on your data
- [ ] generate a 3d plot
- [ ] call a function from Python's ScikitLearn

# 🥳 One cool finding

All dimensionality reduction techniques we used seemed to agree on that European and Japanese cars seem to be similar in specifications where as American cars seem to form their own two clusters based on their specifications.

Blue are American cars. Green and orange are Japanese and European.

<img src="data/0401.png" width="400">


## Clustering
Put simply, the task of clustering is to place observations that seem similar within the same cluster. Clustering is commonly used in two dimensional data where the goal is to create clusters based on coordinates. Here, we will use something similar. We will cluster houses based on their latitude-longitude locations using several different clustering methods.

In [ ]:
# Packages we will use throughout this notebook
import Pkg;
Pkg.add("Clustering")
Pkg.add("VegaLite")
Pkg.add("VegaDatasets")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("JSON")
Pkg.add("CSV")
Pkg.add("Distances")

using Clustering
using VegaLite
using VegaDatasets
using DataFrames
using Statistics
using JSON
using CSV
using Distances

In [ ]:
download("https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv","newhouses.csv")
houses = CSV.read("newhouses.csv",DataFrame)

In [ ]:
names(houses)

We will use the `VegaLite` package here for plotting. This package makes it very easy to plot information on a map. All you need is a JSON file of the map you intend to draw. Here, we will use the California counties JSON file and plot each house on the map and color code it via a heatmap of the price. This is done by this line `color="median_house_value:q"`


In [ ]:
cali_shape = JSON.parsefile("data/california-counties.json")
VV = VegaDatasets.VegaJSONDataset(cali_shape,"data/california-counties.json")

@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="median_house_value:q"
                    
)

One thing we will try and explore in this notebook is if clustering the houses has any direct relationship with their prices, so we will bucket the houses into intervals of $50000 and re perform the color codes based on each bucket.

In [ ]:
bucketprice = Int.(div.(houses[!,:median_house_value],50000))
insertcols!(houses,3,:cprice=bucketprice)

@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="cprice:n"
                    
)

### 🟤K-means clustering

In [ ]:
X = houses[!, [:latitude,:longitude]]
C = kmeans(Matrix(X)', 10) 
insertcols!(houses,3,:cluster10=>C.assignments)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="cluster10:n"
                    
)

Yes, location affects price of the house but this means location as in proximity to water, prosimity to downtown, promisity to a bus stop and so on

lets' see if this remains true for the rest.

### 🟤K-medoids clustering
For this type of clustering, we need to build a distance matrix. We will use the `Distances` package for this purpose and compute the pairwise Euclidean distances.

In [ ]:
xmatrix = Matrix(X)'
D = pairwise(Euclidean(), xmatrix, xmatrix,dims=2) 

K = kmedoids(D,10)
insertcols!(houses,3,:medoids_clusters=>K.assignments)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="medoids_clusters:n"
                    
)

### 🟤Hierarchial Clustering

In [ ]:
K = hclust(D)
L = cutree(K;k=10)
insertcols!(houses,3,:hclust_clusters=>L)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="hclust_clusters:n"
                    
)

### 🟤DBscan

In [ ]:
?dbscan

In [ ]:
using Distances
dclara = pairwise(SqEuclidean(), Matrix(X)',dims=2)
L = dbscan(dclara, 0.05, 10)
@show length(unique(L.assignments))

In [ ]:
insertcols!(houses,3,:dbscanclusters3=>L.assignments)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
    
        fill=:black,
        stroke=:white
    },
    data={
        values=VV,
        format={
            type=:topojson,
            feature=:cb_2015_california_county_20m
        }
    },
    projection={type=:albersUsa},
)+
@vlplot(
    :circle,
    data=houses,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=12},
    color="dbscanclusters3:n"
                    
)

# Finally...
After finishing this notebook, you should be able to:
- [ ] run kmeans clustering on your data
- [ ] run kmedoids clustering on your data
- [ ] run hierarchial clustering on your data
- [ ] run DBscan clustering on your data
- [ ] modify a dataframe and add a new named column
- [ ] generate good looking plots of maps using the VegaLite package

# 🥳 One cool finding

Prices in California do not seem to have an exact mapping with geographical locations. In specifc, performing a clustering algorithm on the houses dataset we had did not reveal a mapping with the price ranges. This indicate that prices relationship to geographical location is not necessairly based on neighborhood but probably other factors like closeness to the water or closeness to a downtown. Here is a figure with a heat map of prices 
<img src="data/0501.png" width="400">
And here is a and k-means clustering of the same houses based on their location
<img src="data/0502.png" width="400">


## Classification
Put simply, classification is the task of predicting a label for a given observation. For example: you are given certain physical descriptions of an animal, and your taks is to classify them as either a dog or a cat. Here, we will classify iris flowers.

As we will see later, we will use different classifiers and at the end of this notebook, we will compare them. We will define our accuracy function right now to get it out of the way. We will use a simple accuracy function that returns the ratio of the number of correctly classified observations to the total number of predictions.

In [ ]:
findaccuracy(predictedvals,groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

In [ ]:
# Packages we will use throughout this notebook
import Pkg;
Pkg.add("GLMNet")
Pkg.add("RDatasets")
Pkg.add("MLBase")
Pkg.add("Plots")
Pkg.add("DecisionTree")
Pkg.add("Distances")
Pkg.add("NearestNeighbors")
Pkg.add("Random")
Pkg.add("LinearAlgebra")
Pkg.add("DataStructures")
Pkg.add("LIBSVM")

using GLMNet
using RDatasets
using MLBase
using Plots
using DecisionTree
using Distances
using NearestNeighbors
using Random
using LinearAlgebra
using DataStructures
using LIBSVM

In [ ]:
iris = dataset("datasets", "iris")

In [ ]:
X = Matrix(iris[:,1:4])
irislabels = iris[:,5]

In [ ]:
X

In [ ]:
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap, irislabels)

In classification, we often want to use some of the data to fit a model, and the rest of the data to validate (commonly known as `training` and `testing` data). We will get this data ready now so that we can easily use it in the rest of this notebook.

In [ ]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at) 
        push!(keepids,rowids...)
    end
    return keepids
end

In [ ]:
?randsubseq

In [ ]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

We will need one more function, and that is the function that will assign classes based on the predicted values when the predicted values are continuous.

In [ ]:
assign_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

### 🟣 Method 1: Lasso

In [ ]:
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])

In [ ]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids])
cv = glmnetcv(X[trainids,:], y[trainids])
mylambda = path.lambda[argmin(cv.meanloss)]

path = glmnet(X[trainids,:], y[trainids],lambda=[mylambda]);

In [ ]:
q = X[testids,:];
predictions_lasso = GLMNet.predict(path,q)

In [ ]:
predictions_lasso = assign_class.(predictions_lasso)
findaccuracy(predictions_lasso,y[testids])

### 🟣 Method 2: Ridge
We will use the same function but set alpha to zero.

In [ ]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0,lambda=[mylambda]);
q = X[testids,:];
predictions_ridge = GLMNet.predict(path,q)
predictions_ridge = assign_class.(predictions_ridge)
findaccuracy(predictions_ridge,y[testids])

### 🟣 Method 3: Elastic Net
We will use the same function but set alpha to 0.5 (it's the combination of lasso and ridge).

In [ ]:
# choose the best lambda to predict with.
path = glmnet(X[trainids,:], y[trainids],alpha=0.5);
cv = glmnetcv(X[trainids,:], y[trainids],alpha=0.5)
mylambda = path.lambda[argmin(cv.meanloss)]
path = glmnet(X[trainids,:], y[trainids],alpha=0.5,lambda=[mylambda]);
q = X[testids,:];
predictions_EN = GLMNet.predict(path,q)
predictions_EN = assign_class.(predictions_EN)
findaccuracy(predictions_EN,y[testids])

### 🟣 Method 4: Decision Trees
We will use the package `DecisionTree`

In [ ]:
model = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

In [ ]:
q = X[testids,:];
predictions_DT = DecisionTree.predict(model, q)
findaccuracy(predictions_DT,y[testids])

### 🟣 Method 5: Random Forests
The `RandomForestClassifier` is available through the `DecisionTree` package as well.

In [ ]:
model = RandomForestClassifier(n_trees=20)
DecisionTree.fit!(model, X[trainids,:], y[trainids])

In [ ]:
q = X[testids,:];
predictions_RF = DecisionTree.predict(model, q)
findaccuracy(predictions_RF,y[testids])

### 🟣 Method 6: Using a Nearest Neighbor method
We will use the `NearestNeighbors` package here.

In [ ]:
Xtrain = X[trainids,:]
ytrain = y[trainids]
kdtree = KDTree(Xtrain')

In [ ]:
queries = X[testids,:]

In [ ]:
idxs, dists = knn(kdtree, queries', 5, true)

In [ ]:
c = ytrain[hcat(idxs...)]
possible_labels = map(i->counter(c[:,i]),1:size(c,2))
predictions_NN = map(i->parse(Int,string(argmax(DataFrame(possible_labels[i])[1,:]))),1:size(c,2))
findaccuracy(predictions_NN,y[testids])

### 🟣 Method 7: Support Vector Machines
We will use the `LIBSVM` package here.

In [ ]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

In [ ]:
model = svmtrain(Xtrain', ytrain)

In [ ]:
predictions_SVM, decision_values = svmpredict(model, X[testids,:]')
findaccuracy(predictions_SVM,y[testids])

In [ ]:
overall_accuracies = zeros(7)
methods = ["lasso","ridge","EN", "DT", "RF","kNN", "SVM"]
ytest = y[testids]
overall_accuracies[1] = findaccuracy(predictions_lasso,ytest)
overall_accuracies[2] = findaccuracy(predictions_ridge,ytest)
overall_accuracies[3] = findaccuracy(predictions_EN,ytest)
overall_accuracies[4] = findaccuracy(predictions_DT,ytest)
overall_accuracies[5] = findaccuracy(predictions_RF,ytest)
overall_accuracies[6] = findaccuracy(predictions_NN,ytest)
overall_accuracies[7] = findaccuracy(predictions_SVM,ytest)
hcat(methods, overall_accuracies)

# Finally...
After finishing this notebook, you should be able to:
- [ ] split your data into training and testing data to test the effectiveness of a certain method
- [ ] apply a simple accuracy function to test the effectiveness of a certain method
- [ ] run multiple classification algorithms:
    - [ ] LASSO
    - [ ] Ridge
    - [ ] ElasticNet
    - [ ] Decision Tree
    - [ ] Random Forest
    - [ ] Nearest Neighbors
    - [ ] Support Vector Machines

# 🥳 One cool finding

We used multiple methods to run classification on the `iris` dataset which is a dataset of flowers and there are three types of iris flowers in it. We split the data into training and testing and ran our methods. Here is the scoreboard:

| method | accuracy score |
|---|---|
| lasso  |1.0|
| ridge  |1.0|
| EN     |1.0|
| DT     |0.960784|
| RF     |0.980392|
| kNN    |1.0|
| SVM    |1.0|

In [ ]:
# Packages we will use throughout this notebook
import Pkg;
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("PyCall")
Pkg.add("DataFrames")
Pkg.add("GLM")
Pkg.add("Tables")
Pkg.add("XLSX")
Pkg.add("MLBase")
Pkg.add("RDatasets")
Pkg.add("LsqFit")

using Plots
using Statistics
using StatsBase
using PyCall
using DataFrames
using GLM
using Tables
using XLSX
using MLBase
using RDatasets
using LsqFit

In [ ]:
xvals = repeat(1:0.5:10, inner=2)
yvals = 3 .+ xvals .+ 2 .* rand(length(xvals)) .-1
scatter(xvals, yvals, color=:black, leg=false)

In [ ]:
function find_best_fit(xvals,yvals)
    meanx = mean(xvals)
    meany = mean(yvals)
    stdx = std(xvals)
    stdy = std(yvals)
    r = cor(xvals,yvals)
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
end

In [ ]:
a,b = find_best_fit(xvals,yvals)
ynew = a .* xvals .+ b

In [ ]:
np = pyimport("numpy");

In [ ]:
xdata = xvals
ydata = yvals
@time myfit = np.polyfit(xdata, ydata, 1);
ynew2 = collect(xdata) .* myfit[1] .+ myfit[2];
scatter(xvals,yvals)
plot!(xvals,ynew)
plot!(xvals,ynew2)

In [ ]:
data = DataFrame(X=xdata, Y=ydata)
ols = lm(@formula(Y ~ X), data)
plot!(xdata,predict(ols))

Now let's get some real data. We will use housing information from zillow, check out the file `zillow_data_download_april2020.xlsx` for a quick look of what the data looks like. Our goal will be to build a linear regression model between the number of houses listed vs the number of houses sold in a few states. Fitting these models can serve as a key real estate indicator.

In [ ]:
# play around with data for a bit
R = XLSX.readxlsx("data/zillow_data_download_april2020.xlsx")

In [ ]:
sale_counts = R["Sale_counts_city"][:]
df_sale_counts = DataFrame(sale_counts[2:end,:],Symbol.(sale_counts[1,:]))

monthly_listings = R["MonthlyListings_City"][:]
df_monthly_listings = DataFrame(monthly_listings[2:end,:],Symbol.(monthly_listings[1,:]))

In [ ]:
monthly_listings_2020_02 = df_monthly_listings[!,[1,2,3,4,5,end]]
rename!(monthly_listings_2020_02, Symbol("2020-02") .=> Symbol("listings"))

sale_counts_2020_02 = df_sale_counts[!,[1,end]]
rename!(sale_counts_2020_02, Symbol("2020-02") .=> Symbol("sales"))

In [ ]:
Feb2020data

In [ ]:
Feb2020data = innerjoin(monthly_listings_2020_02,sale_counts_2020_02,on=:RegionID) #, type="outer")
dropmissing!(Feb2020data)
sales = Feb2020data[!,:sales]
# prices = Feb2020data[!,:price]
counts = Feb2020data[!,:listings]
using DataStructures
states = Feb2020data[!,:StateName]
C = counter(states)
C.map
countvals = values(C.map)
topstates = sortperm(collect(countvals),rev=true)[1:10]
states_of_interest = collect(keys(C.map))[topstates]
all_plots = Array{Plots.Plot}(undef,10)

In [ ]:
Feb2020data

In [ ]:
all_plots = Array{Plots.Plot}(undef,10)
for (i,si) in enumerate(states_of_interest)
    curids = findall(Feb2020data[!,:StateName].==si)
    data = DataFrame(X=float.(counts[curids]), Y=float.(sales[curids]))
    ols = GLM.lm(@formula(Y ~ 0 + X), data)    
    all_plots[i] = scatter(counts[curids],sales[curids],markersize=2,
        xlim=(0,500),ylim=(0,500),color=i,aspect_ratio=:equal,
        legend=false,title=si)
    @show si,coef(ols)
    plot!(counts[curids],predict(ols),color=:black)
end
plot(all_plots...,layout=(2,5),size=(900,300))

In [ ]:
all_plots = Array{Plots.Plot}(undef,10)
for (i,si) in enumerate(states_of_interest)
    curids = findall(Feb2020data[!,:StateName].==si)
    data = DataFrame(X=float.(counts[curids]), Y=float.(sales[curids]))
    ols = GLM.lm(@formula(Y ~ X), data)    
    all_plots[i] = scatter(counts[curids],sales[curids],markersize=2,
        xlim=(0,500),ylim=(0,500),color=i,aspect_ratio=:equal,
        legend=false,title=si)
    @show si,coef(ols)
    plot!(counts[curids],predict(ols),color=:black)
end
plot(all_plots...,layout=(2,5),size=(900,300))

In [ ]:
plot()
for (i,si) in enumerate(states_of_interest)
    curids = findall(Feb2020data[!,:StateName].==si)
    data = DataFrame(X=float.(counts[curids]), Y=float.(sales[curids]))
    ols = GLM.lm(@formula(Y ~ 0 + X), data)    
    scatter!(counts[curids],sales[curids],markersize=2,
        xlim=(0,500),ylim=(0,500),color=i,aspect_ratio=:equal,
        legend=false,marker=(3,3,stroke(0)),alpha=0.2)
        if si == "NC" || si == "CA" || si == "FL"
            annotate!([(500-20,10+coef(ols)[1]*500,text(si,10))])
        end
    @show si,coef(ols)
    plot!(counts[curids],predict(ols),color=i,linewidth=2)
end
# plot(all_plots...,layout=(2,5),size=(900,300))
xlabel!("listings")
ylabel!("sales")

---- 
### 🟠 Logistic regression
So far, we have shown several ways to solve the linear regression problem in Julia. Here, we will first start with a motivating example of when you would want to use logistic regression. Let's assume that our predictor vector is binary (`0` or `1`), let's fit a linear regression model.

In [ ]:
data = DataFrame(X=[1,2,3,4,5,6,7], Y=[1,0,1,1,1,1,1])
linear_reg = lm(@formula(Y ~ X), data)
scatter(data[!,:X],data[!,:Y],legend=false,size=(300,200))
plot!(1:7,predict(linear_reg))

What this plot quickly shows is that linear regression may end up predicting values outside the `[0,1]` interval. For an example like this, we will use logistic regression. Interestingly, a generalized linear model (https://en.wikipedia.org/wiki/Generalized_linear_model) unifies concepts like linear regression and logistic regression, and the `GLM` package allows you to apply either of these regressions easily by specifying the `distribution family` and the `link` function. 

To apply logistic regression via the `GLM` package, you can readily use the `Binomial()` family and the `LogitLink()` link function. 

Let's load some data and take a look at one example.

In [ ]:
# we will load this data from RDatasets
cats = dataset("MASS", "cats")

In [ ]:
lmap = labelmap(cats[!,:Sex])
ci = labelencode(lmap, cats[!,:Sex])
scatter(cats[!,:BWt],cats[!,:HWt],color=ci,legend=false)

In [ ]:
lmap

In [ ]:
data = DataFrame(X=cats[!,:HWt], Y=ci.-1)
probit = glm(@formula(Y ~ X), data, Binomial(), LogitLink())
scatter(data[!,:X],data[!,:Y],label="ground truth gender",color=6)
scatter!(data[!,:X],predict(probit),label="predicted gender",color=7)

-----
### 🟠 Non linear regression
Finally, sometimes you may have a set of points and the goal is to fit a non-linear function (maybe a quadratic function, a cubic function, an exponential function...). The way we would solve such a problem is by minimizing the least square error between the fitted function and the observations we have. We will use the package `LsqFit` for this task. Note that this problem is usually modeled as a numerical optimizaiton problem.

We will first set up our data.

In [ ]:
xvals = 0:0.05:10
yvals = 1*exp.(-xvals*2) + 2*sin.(0.8*pi*xvals) + 0.15 * randn(length(xvals));
scatter(xvals,yvals,legend=false)

In [ ]:
@. model(x, p) = p[1]*exp(-x*p[2]) + p[3]*sin(0.8*pi*x)
p0 = [0.5, 0.5, 0.5]
myfit = curve_fit(model, xvals, yvals, p0)

In [ ]:
p = myfit.param
findyvals = p[1]*exp.(-xvals*p[2]) + p[3]*sin.(0.8*pi*xvals)
scatter(xvals,yvals,legend=false)
plot!(xvals,findyvals)

In [ ]:
@. model(x, p) = p[1]*x
myfit = curve_fit(model, xvals, yvals, [0.5])
p = myfit.param
findyvals = p[1]*xvals
scatter(xvals,yvals,legend=false)
plot!(xvals,findyvals)

# Finally...
After finishing this notebook, you should be able to:
- [ ] run a linear regression model
- [ ] use the GLM package to pass functions and probability distributions to solve your special regression problem
- [ ] use GLM to solve a logistic regression problem
- [ ] fit a nonlinear regression to your data using the LsqFit package
- [ ] use the LsqFit package to fit a linear function too

# 🥳 One cool finding

One metric in real estate is to find the number of houses being sold out of the number of houses on the market. We collect multiple data points from multiple states and fit a linear model to these states. It turns out that North Carolina has the highest sold/listed ratios. Florida is one of the least, and California is somewhere in between.

<img src="data/0701.png" width="400">


In [ ]:
# Packages we will use throughout this notebook
import Pkg;
Pkg.add("LightGraphs")
Pkg.add("MatrixNetworks")
Pkg.add("VegaDatasets")
Pkg.add("DataFrames")
Pkg.add("SparseArrays")
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("VegaLite")
Pkg.add("MatrixNetworks")
using MatrixNetworks
using LightGraphs
using MatrixNetworks
using VegaDatasets
using DataFrames
using SparseArrays
using LinearAlgebra
using Plots
using VegaLite

In [ ]:
data = VegaDatasets.dataset("cars")

In [ ]:
airports = VegaDatasets.dataset("airports")
flightsairport = VegaDatasets.dataset("flights-airport")

In [ ]:
airports

In [ ]:
flightsairport

In [ ]:
flightsairportdf = DataFrame(flightsairport)

In [ ]:
allairports = vcat(flightsairportdf[!,:origin],flightsairportdf[!,:destination])

In [ ]:
uairports = unique(allairports)

In [ ]:
# create an airports data frame that has a subset of airports that are only included in the routes dataset
airportsdf = DataFrame(airports)
subsetairports = map(i->findfirst(airportsdf[!, :iata].==uairports[i]),1:length(uairports))
airportsdf_subset = airportsdf[subsetairports,:]

In [ ]:
using VegaLite, VegaDatasets
p1 = @vlplot(data=VegaDatasets.dataset("cars"),
    mark={type="point", tooltip=true, size=75},
    x=:Horsepower,
    y=:Miles_per_Gallon,
    color={condition={selection="org", field="Cylinders", type="ordinal"}, value=""},
    width=800,
    height=600,
    selection={org={type="single", fields=["Origin"], bind={input="select", options=[nothing, "Europe", "Japan", "USA"]}}},
    hover=:Miles_per_Gallon)

save("cars_dropdown.html", p1)

In [ ]:
# build the adjacency matrix
ei_ids = findfirst.(isequal.(flightsairportdf[!,:origin]), [uairports])
ej_ids = findfirst.(isequal.(flightsairportdf[!,:destination]), [uairports])
edgeweights = flightsairportdf[!,:count];

In [ ]:
A = sparse(ei_ids,ej_ids,1,length(uairports),length(uairports))
A = max.(A,A')

In [ ]:
StatsPlots.spy(A)

In [ ]:
LinearAlgebra.issymmetric(A)

In [ ]:
L = SimpleGraph(A)

In [ ]:
G=SimpleGraph(10) #SimpleGraph(nnodes,nedges) 
add_edge!(G,7,5)#modifies graph in place.
add_edge!(G,3,5)
add_edge!(G,5,2)

In [ ]:
cc = scomponents(A)

In [ ]:
degrees = sum(A,dims=2)[:]
p1 = Plots.plot(sort(degrees,rev=true),ylabel="log degree",legend=false,yaxis=:log)
p2 = Plots.plot(sort(degrees,rev=true),ylabel="degree",legend=false)
Plots.plot(p1,p2,size=(600,300))

This is actually very interesting because it looks like that the airline transportation network seems to fit a powerlaw degree distribution. Knowing that your graph fits a well known model for degree distribution can be very helpful for further studying it. (For instance, there is a lot of literature on graphs that fit power law degree distributions).

In [ ]:
maxdegreeid = argmax(degrees)
uairports[maxdegreeid]

In [ ]:
us10m = VegaDatasets.dataset("us-10m")
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill=:lightgray,
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=10},
    color={value=:steelblue}
)+
@vlplot(
    :rule,
    data=flightsairport,
    transform=[
        {filter={field=:origin,equal=:ATL}},
        {
            lookup=:origin,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["origin_latitude", "origin_longitude"]
        },
        {
            lookup=:destination,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["dest_latitude", "dest_longitude"]
        }
    ],
    projection={type=:albersUsa},
    longitude="origin_longitude:q",
    latitude="origin_latitude:q",
    longitude2="dest_longitude:q",
    latitude2="dest_latitude:q"
)

### 🟡Shortest path problem
Finding the shortest path between two nodes. We will use Dijkstra's algorithm.

In [ ]:
ATL_paths = dijkstra(A,maxdegreeid)

In [ ]:
ATL_paths[1][maxdegreeid]

In [ ]:
maximum(ATL_paths[1])

In [ ]:
@show stop1 = argmax(ATL_paths[1])
@show uairports[stop1];

In [ ]:
@show stop2 = ATL_paths[2][stop1]
@show uairports[stop2];

In [ ]:
@show stop3 = ATL_paths[2][stop2]
@show uairports[stop3];

In [ ]:
@show stop4 = ATL_paths[2][stop3]
@show uairports[stop4];

In [ ]:
using VegaLite, VegaDatasets

us10m = VegaDatasets.dataset("us-10m")
airports = VegaDatasets.dataset("airports")

@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=5},
    color={value=:gray}
) +
@vlplot(
    :line,
    data={
        values=[
            {airport=:ATL,order=1},
            {airport=:SEA,order=2},
            {airport=:JNU,order=3},
            {airport=:GST,order=4}
        ]
    },
    transform=[{
        lookup=:airport,
        from={
            data=airports,
            key=:iata,
            fields=["latitude","longitude"]
        }
    }],
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    order={field=:order,type=:ordinal}
)

In [ ]:
nodeid = argmin(degrees)
@show uairports[nodeid]
d = dijkstra(A,nodeid)
argmax(d[1]),uairports[argmax(d[1])]

In [ ]:
function find_path(d,id)
    shortestpath = zeros(Int,1+Int.(d[1][id]))
    shortestpath[1] = id
    for i = 2:length(shortestpath)
        shortestpath[i] = d[2][shortestpath[i-1]]
    end
    return shortestpath
end
p = find_path(d,123)
uairports[p]

### 🟡Minimum Spanning Tree (MST)
The next problem is forming a minimum spanning tree on the graph. The idea of a minimum spanning tree is to connect all nodes in the graph with as little edges as possible. We will use Prim's algorithm for this problem.

In [ ]:
?mst_prim

In [ ]:
ti,tj,tv,nverts = mst_prim(A)

In [ ]:
df_edges = DataFrame(:ei=>uairports[ti],:ej=>uairports[tj])

In [ ]:
@vlplot(width=800, height=500) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size={value=20},
    color={value=:gray}
) +
@vlplot(
    :rule,
    data=df_edges, #data=flightsairport,
    transform=[
        {
            lookup=:ei,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["originx", "originy"]
        },
        {
            lookup=:ej,
            from={
                data=airportsdf_subset,
                key=:iata,
                fields=["latitude", "longitude"]
            },
            as=["destx", "desty"]
        }
    ],
    projection={type=:albersUsa},
    longitude="originy:q",
    latitude="originx:q",
    longitude2="desty:q",
    latitude2="destx:q"
)

### 🟡PageRank
PageRank is the algorithm that got Google started. The idea is: given an network of connections between multiple nodes (web pages in the cae of Google), is there a way to return a list of ranked nodes? PageRank provides this ranking. Obviously, nodes can be ranked in several different ways but PageRank remains to be one of the most popular methods in network analysis. Let's check out the documentation of `pagerank` below.

In [ ]:
?MatrixNetworks.pagerank

In [ ]:
v = MatrixNetworks.pagerank(A,0.85)

In [ ]:
sum(v)

In [ ]:
insertcols!(airportsdf_subset,7,:pagerank_value=>v)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size="pagerank_value:q",
    color={value=:steelblue}
)

### 🟡Clustering Coefficients
From Wikipedia: The local clustering coefficient of a vertex (node) in a graph quantifies how close its neighbours are to being a clique (complete graph).

This means that if for example, a node is connected to two nodes that are also connected to each other, that node's clustering coeefficient is 1. This can be a good metric to find out which nodes tend to have tight clusters around them. Let's look at the documentation of `clustercoeffs` from MatrixNetworks.

In [ ]:
?clustercoeffs

In [ ]:
cc = clustercoeffs(A)
cc[findall(cc.<=eps())] .= 0
cc

In [ ]:
insertcols!(airportsdf_subset,7,:ccvalues=>cc)

In [ ]:
@vlplot(width=500, height=300) +
@vlplot(
    mark={
        :geoshape,
        fill="#eee",
        stroke=:white
    },
    data={
        values=us10m,
        format={
            type=:topojson,
            feature=:states
        }
    },
    projection={type=:albersUsa},
) +
@vlplot(
    :circle,
    data=airportsdf_subset,
    projection={type=:albersUsa},
    longitude="longitude:q",
    latitude="latitude:q",
    size="ccvalues:q",
    color={value=:gray}
)

# Finally...
After finishing this notebook, you should be able to:
- [ ] take a list of connections between nodes and form an adjacency matrix out of them
- [ ] use the LightGraphs package to create a graph
- [ ] detect if a graph is connected or not
- [ ] solve the shortest path problem on a graph and a given node
- [ ] solve the minimum spanning tree problem on a graph
- [ ] solve the PageRank problem on a graph
- [ ] find the clustering coefficients of nodes in a graph

# 🥳 One cool finding

We ran PageRank (which is a common algorithm to rank nodes in a network), and visualized the PageRank values on the US airports. The results agreed with the hypothesis that known hubs have higher PageRank value. Look at Atlanta, it's actually the biggest.

<img src="data/0801.png" width="600">


In [ ]:
# Numerical Optimization

In [ ]:
# Packages we will use throughout this notebook
import Pkg;
Pkg.add("Convex")
Pkg.add("SCS")
Pkg.add("XLSX")
Pkg.add("DataFrames")
Pkg.add("Plots")
Pkg.add("CSV")
Pkg.add("Statistics")
Pkg.add("Images")
Pkg.add("DelimitedFiles")
using Pkg
using Convex
using SCS
using XLSX
using DataFrames
using Plots
using CSV
using Statistics
using Images
using DelimitedFiles

### 📈 Problem 1 Portfolio investment.
Our first problem will be an investment problem. We will look at stock prices from three companies and decide how to spend an amount of $1000 on these three companies. Let's first load some data.

In [ ]:
T = DataFrame(XLSX.readtable("data/stock_prices.xlsx","Sheet2")...)

`T` is a `DataFrame` that has weekly stock prices value of three companies (Microsoft, Facebook, Apple) from the period of January 2019 - March 2019. We will first take a quick look at these prices in a quick plot.

In [ ]:
plot(T[!,:MSFT],label="Microsoft")
plot!(T[!,:AAPL],label="Apple")
plot!(T[!,:FB],label="FB")

In [ ]:
# convert the prices to a Matrix to be used later in the optimization problem
prices_matrix = Matrix(T)

In [ ]:
M1 = prices_matrix[1:end-1,:]
M2 = prices_matrix[2:end,:]
R = (M2.-M1)./M1

Now let's assume that the vector `x = [x1 x2 x3]` will contain the total number of dollars we will invest in these companies, i.e. `x1` is how much we will invest in the first company (MSFT), `x2` is how much we will invest in FB, and `x3` is how much we will invest in `AAPL`. The return on the investment will be `dot(r,x)`, where `r = [r1 r2 r3]` is the return from each of the companies.

Here, `r` is a random variable and we will have to model it in terms of expected values. And the expected value `E(dot(r,x))` will be `E[dot(mean(R,dims=2),x)`. If we want a return of `10%` or more, then we need `dot(r,x) >= 0.1`.

Next, we will model the risk matrix. We will skip the derivation of the risk matrix here, but you can read about it here: https://www.kdnuggets.com/2019/06/optimization-python-money-risk.html. The risk matrix will be the covariance matrix of the computed return prices (`R`).

In [ ]:
risk_matrix = cov(R)

In [ ]:
# note that the risk matrix is positive definite
isposdef(risk_matrix)

In [ ]:
r = mean(R,dims=1)[:]

Now let's solve the following problem: Someone gives you $1000 and tells you to spend them in the form of investment on these three compnaies such that you get a return of 2\% on what you spent.

The goal will be to minimize the risk, that is x'\*risk_matrix\*x.
The constraints will be 
- `sum(x) = 1`, we will compute the percentage of investment rather than the exact amount
- `dot(r,x) >= 0.02`
- `x[i] >= 0`

This problem is a convext problem, and we will use `Convex.jl` to it.

In [ ]:
x = Variable(length(r))
problem = minimize(x'*risk_matrix*x,[sum(x)==1;r'*x>=0.02;x.>=0])

Note the `Convex.NotDcp` in the answer above and the warning. `Convex.jl` requires that we pass Dcp compliant problem (Disciplined convex programming). Learn more about the DCP ruleset here: http://cvxr.com/cvx/doc/dcp.html

In [ ]:
# make the problem DCP compliant
problem = minimize(Convex.quadform(x,risk_matrix),[sum(x)==1;r'*x>=0.02;x.>=0])

In [ ]:
solve!(problem, SCS.Optimizer)

In [ ]:
x

In [ ]:
sum(x.value)

In [ ]:
# return
r'*x.value

In [ ]:
x.value .* 1000

In [ ]:
The conclusion is that we should invest **67.9USD in Microsoft**, **122.3USD in Facebook**, and **809.7USD in Apple**.

---
### 🖼️ Problem 2 Image recovery.
In this problem, we are given an image where some of the pixels have been altered. The goal is to recover the unknonwn pixels by solving an optimization problem. Let's first load the figure.

In [ ]:
Kref = load("data/khiam-small.jpg")

In [ ]:
K = copy(Kref)
p = prod(size(K))
missingids = rand(1:p,400)
K[missingids] .= RGBX{N0f8}(0.0,0.0,0.0)
K
Gray.(K)

In [ ]:
Y = Float64.(Gray.(K));

Given this image, the goal is now to complete the matrix. We will use a common technique for this problem developed by Candes and Tao. The goal will be to create a new matrix `X` where we minimize the nuclear norm of `X` (i.e. the sum of the singular values of `X`), and such that the entries that are already known in `Y` remain the same in `X`. We will again use `Convex.jl` to solve this problem. Let's write it down below.

In [ ]:
correctids = findall(Y[:].!=0)
X = Convex.Variable(size(Y))
problem = minimize(nuclearnorm(X))
problem.constraints += X[correctids]==Y[correctids]

In [ ]:
solve!(problem, SCS.Optimizer(eps=1e-3, alpha=1.5))

In [ ]:
@show norm(float.(Gray.(Kref))-X.value)
@show norm(-X.value)
colorview(Gray, X.value)

In [ ]:
@show norm(float.(Gray.(Kref))-X.value)
@show norm(-X.value)
colorview(Gray, X.value)

---
### 🥒 Problem 3 Diet optimization problem.
This is a common problem in Numerical Optimization, and you can find multiple references about it online. Here, we will use one of the examples in the JuMP package. Refer to this page for details: https://github.com/JuliaOpt/JuMP.jl/blob/master/examples/diet.jl.

In this porblem we are given constraints on the number of (minimum, maximum) number of calories, protein, fat, and sodium to consume. We will first build a JuMP container to store this information and pass it as constraints later.

In [ ]:
import Pkg
Pkg.add("JuMP")
Pkg.add("GLPK")
using JuMP
using GLPK

In [ ]:
category_data = JuMP.Containers.DenseAxisArray(
    [1800 2200;
     91   Inf;
     0    65;
     0    1779], 
    ["calories", "protein", "fat", "sodium"], 
    ["min", "max"])

You can think of this matrix as indexed by rows via the vector `["calories", "protein", "fat", "sodium"]`, and indexed by columns via the vector `["min", "max"]`. In fact, we can now checkout the values: `category_data["calories","max"]` or `category_data["fat","min"]`.

In [ ]:
@show category_data["calories","max"] 
@show category_data["fat","min"];

In [ ]:
foods = ["hamburger", "chicken", "hot dog", "fries", "macaroni", "pizza","salad", "milk", "ice cream"]

# we will use the same concept we used above to create an array indexed 
# by foods this time to record the cost of each of these items
cost = JuMP.Containers.DenseAxisArray(
    [2.49, 2.89, 1.50, 1.89, 2.09, 1.99, 2.49, 0.89, 1.59],
    foods)

Next we will create a new matrix to encode the calories,protein, fat, and sodium present in each of these foods. This will be a matrix encoded by foods by rows, and `["calories", "protein", "fat", "sodium"]` by columns.

In [ ]:
food_data = JuMP.Containers.DenseAxisArray(
    [410 24 26 730;
     420 32 10 1190;
     560 20 32 1800;
     380  4 19 270;
     320 12 10 930;
     320 15 12 820;
     320 31 12 1230;
     100  8 2.5 125;
     330  8 10 180], 
    foods, 
    ["calories", "protein", "fat", "sodium"])

@show food_data["chicken", "fat"]
@show food_data["milk", "sodium"];

In [ ]:
# set up the model
model = Model(GLPK.Optimizer)

categories = ["calories", "protein", "fat", "sodium"]

# add the variables
@variables(model, begin
    # Variables for nutrition info
    category_data[c, "min"] <= nutrition[c = categories] <= category_data[c, "max"]
    # Variables for which foods to buy
    buy[foods] >= 0
end)

# Objective - minimize cost
@objective(model, Min, sum(cost[f] * buy[f] for f in foods))

# Nutrition constraints
@constraint(model, [c in categories],
    sum(food_data[f, c] * buy[f] for f in foods) == nutrition[c]
)

In [ ]:
JuMP.optimize!(model)
term_status = JuMP.termination_status(model)
is_optimal = term_status == MOI.OPTIMAL
@show JuMP.primal_status(model) == MOI.FEASIBLE_POINT
@show JuMP.objective_value(model) ≈ 11.8288 atol = 1e-4

In [ ]:
hcat(buy.data,JuMP.value.(buy.data))

----
### 🗺️ How many passports do you need to travel the world without obtaining a visa in advance?
This problem is the same problem shown in the JuliaCon 2018 JuMP workshop, with updated code and data. The original post can be found here: https://github.com/juan-pablo-vielma/JuliaCon2018_JuMP_Workshop/blob/master/Introduction_Slides.ipynb.

We will first get the data.

In [ ]:
;git clone https://github.com/ilyankou/passport-index-dataset.git

The file we need is `passport-index-dataset/passport-index-matrix.csv`, and we will use the `DelimitedFiles` package to read it -- this is mainly because what we are loading is a matrix and we will have to extract the matrix out of the DataFrame if we use the `CSV` package. Both are viable options, this will just be quicker.

In [ ]:
passportdata = readdlm(joinpath("passport-index-dataset","passport-index-matrix.csv"),',')

In [ ]:
cntr = passportdata[2:end,1]
vf = (x ->  typeof(x)==Int64 || x == "VF" || x == "VOA" ? 1 : 0).(passportdata[2:end,2:end]);

In [ ]:
model = Model(GLPK.Optimizer)

In [ ]:
@variable(model, pass[1:length(cntr)], Bin)
@constraint(model, [j=1:length(cntr)], sum( vf[i,j]*pass[i] for i in 1:length(cntr)) >= 1)
@objective(model, Min, sum(pass))

In [ ]:
JuMP.optimize!(model)

In [ ]:
print(JuMP.objective_value(model)," passports: ",join(cntr[findall(JuMP.value.(pass) .== 1)],", "))

# Finally...
After finishing this notebook, you should be able to:
- [ ] solve optimization problems via the Convex.jl package
- [ ] solve optimization problems via the JuMP.jl package

# 🥳 One cool finding

We found out that you need at least 21 passports to tour the world visa free. Here is one solutions we found:

`Afghanistan, Austria, Comoros, Equatorial Guinea, Eritrea, Gambia, Georgia, Hong Kong, India, Iraq, Kenya, Madagascar, Maldives, North Korea, Papua New Guinea, Seychelles, Singapore, Somalia, Tunisia, United Arab Emirates, Zimbabwe`


## Neural Networks
In this notebook, we will walk through one main neural nets example. And that is, classifying the infamous MNIST dataset. **If you have no experience with neural nets prior to this notebook, I recommend doing a quick search for an "intro to neural nets"**, there are multiple tutorials/blog posts out there and you can choose the one that works for you.

Here, we will use the `Flux` package, but if you want to look at other packages I encourage you to look at `Knet.jl` and `TensorFlow.jl`.

In [ ]:
import Pkg; 
Pkg.add("Flux")
Pkg.add("Images")
using Flux, Flux.Data.MNIST
using Flux: onehotbatch, argmax, crossentropy, throttle
using Base.Iterators: repeated
using Images

In [ ]:
imgs = MNIST.images()
colorview(Gray, imgs[100])

In [ ]:
typeof(imgs[3])

In [ ]:
myFloat32(X) = Float32.(X)
fpt_imgs = myFloat32.(imgs) 

In [ ]:
typeof(fpt_imgs[3])

In [ ]:
vectorize(x) = x[:]
vectorized_imgs = vectorize.(fpt_imgs);

In [ ]:
typeof(vectorized_imgs)

In [ ]:
X = hcat(vectorized_imgs...)
size(X)

In [ ]:
onefigure = X[:,3]
t1 = reshape(onefigure,28,28)
colorview(Gray,t1)

In [ ]:
labels = MNIST.labels()
labels[1]

In [ ]:
Y = onehotbatch(labels, 0:9)

In [ ]:
m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax)

In [ ]:
m(onefigure)

In [ ]:
loss(x, y) = Flux.crossentropy(m(x), y)
accuracy(x, y) = mean(argmax(m(x)) .== argmax(y))

In [ ]:
datasetx = repeated((X, Y), 200)
C = collect(datasetx);

In [ ]:
evalcb = () -> @show(loss(X,Y))

In [ ]:
ps = Flux.params(m)

In [ ]:
?Flux.train!

In [ ]:
opt = ADAM()
Flux.train!(loss, ps, datasetx, opt, cb = throttle(evalcb, 10))

In [ ]:
tX = hcat(float.(reshape.(MNIST.images(:test), :))...);
test_image = m(tX[:,1])

In [ ]:
argmax(test_image) - 1

In [ ]:
t1 = reshape(tX[:,1],28,28)
colorview(Gray, t1)

In [ ]:
onefigure = X[:,2]
m(onefigure)

In [ ]:
Y[:,2]

# Finally...
After finishing this notebook, you should be able to:
- [ ] prepare data to fit the format to create a neural network using Flux.jl
- [ ] create a neural network with Flux.jl
- [ ] creating an accuracy function and loss function to be passed to train the neural network
- [ ] train the neural network
- [ ] describe a few tips that can help make your nerual network faster or more accurate (such as using Float32 as opposed to Float32)

# 🥳 One cool finding

We ran a trained a neural network on a dataset of of handwritten digits (called the MNIST dataset). At the end, we were able to pass this figure to the neural network and the return result was:

<img src="data/1001.png" width="40">

```
10-element Array{Float32,1}:
 0.00029263002
 1.5993925f-5
 0.0002862561
 0.0035434738
 1.388653f-5
 2.4878627f-5
 6.433018f-7
 0.99414164 ### <= this is the highest number!
 0.000118321994
 0.0015623316
```

## Using other languages
Often, I hear that the biggest challenge of moving from another language to Julia is giving up all the codes you have written in other languages or your favorite packages from other languages. **This notebook is not about data science, but it's about your next data science project** (if you're working on a data science project in Julia and you want to use functionality from other langages). Here, we will specifically cover Python, R, and C.

### ⚫Python

In [ ]:
import Pkg
Pkg.add("PyCall")
using PyCall

In [ ]:
math = pyimport("math")
math.sin(math.pi / 4)
# returns ≈ 1/√2 = 0.70710678...

In [ ]:
python_networkx = pyimport("networkx")

In [ ]:
py"""
import numpy
def find_best_fit_python(xvals,yvals):
    meanx = numpy.mean(xvals)
    meany = numpy.mean(yvals)
    stdx = numpy.std(xvals)
    stdy = numpy.std(yvals)
    r = numpy.corrcoef(xvals,yvals)[0][1]
    a = r*stdy/stdx
    b = meany - a*meanx
    return a,b
"""

In [ ]:
xvals = repeat(1:0.5:10, inner=2)
yvals = 3 .+ xvals .+ 2 .* rand(length(xvals)) .-1
find_best_fit_python = py"find_best_fit_python"
a,b = find_best_fit_python(xvals,yvals)

If the above python code was in a file called `fit_linear.py`, you can call it as follows:
```
python_linear_fit = pyimport("fit_linear") 
python_linear_fit.find_best_fit_python(xvals,yvals)```

### ⚫R code

In [ ]:
import Pkg
Pkg.add("RCall")
using RCall

In [ ]:
# we can use the rcall function
r = rcall(:sum, Float64[1.0, 4.0, 6.0])

In [ ]:
typeof(r[1])

In [ ]:
z = 1
@rput z

In [ ]:
r = R"z+z"

In [ ]:
r[1]

In [ ]:
x = randn(10)

In [ ]:
@rimport base as rbase
rbase.sum([1,2,3])

In [ ]:
@rlibrary boot

In [ ]:
using HypothesisTests
OneSampleTTest(x)

### ⚫C code
Calling standard libraries is easy

In [ ]:
t = ccall(:clock, Int32, ())

Can look at Python and C/C++ examples here: https://github.com/xorJane/Excelling-at-Julia-Basics-and-Beyond/blob/master/JuliaCon2019_Huda/Julia%20Wrappers.ipynb
```
ccall((:hello_world_repeated,"hello_world_lib.dylib"),
    Int64,
    (Int64,),
    10)
    ```
    
**Finally**, I would say that this is the only off-topic notebook in this course, and it's a topic that can be covered on its own in a standalone tutorial... Nevertheless, the goal of this notebook is to tell you that porting your code from Python, R, and C should be easy and straight forward in Julia. 

# 🥳 One cool finding

You can easily call Python, R, C, and Cpp code from Julia!

## Visualization
In this notebook, we will cover five visualizations and the hope is that each of them will reveal tips for you for your next plot. There is a wide range of visually pleasing plots you can generate with Julia and I strongly recommend envisioning what you want to plot beforehand and then figuring out how to accomplish it (most likely you will find all what you need in at least one julia plotting package). Here, we will be using the `Plots` package with a `gr()` backend

In [ ]:
ENV["GKS_ENCODING"] = "utf-8"

In [ ]:
stateabbreviations = Dict("Alabama" => "AL",
    "Alaska" => "AK",
    "Arizona" => "AZ",
    "Arkansas" => "AR",
    "California" => "CA",
    "Colorado" => "CO",
    "Connecticut" => "CT",
    "Delaware" => "DE",
    "Florida" => "FL",
    "Georgia" => "GA",
    "Hawaii" => "HI",
    "Idaho" => "ID",
    "Illinois" => "IL",
    "Indiana" => "IN",
    "Iowa" => "IA",
    "Kansas" => "KS",
    "Kentucky" => "KY",
    "Louisiana" => "LA",
    "Maine" => "ME",
    "Maryland" => "MD",
    "Massachusetts" => "MA",
    "Michigan" => "MI",
    "Minnesota" => "MN",
    "Mississippi" => "MS",
    "Missouri" => "MO",
    "Montana" => "MT",
    "Nebraska" => "NE",
    "Nevada" => "NV",
    "New Hampshire" => "NH",
    "New Jersey" => "NJ",
    "New Mexico" => "NM",
    "New York" => "NY",
    "North Carolina" => "NC",
    "North Dakota" => "ND",
    "Ohio" => "OH",
    "Oklahoma" => "OK",
    "Oregon" => "OR",
    "Pennsylvania" => "PA",
    "Rhode Island" => "RI",
    "South Carolina" => "SC",
    "South Dakota" => "SD",
    "Tennessee" => "TN",
    "Texas" => "TX",
    "Utah" => "UT",
    "Vermont" => "VT",
    "Virginia" => "VA",
    "Washington" => "WA",
    "West Virginia" => "WV",
    "Wisconsin" => "WI",
    "Wyoming" => "WY", 
    "District of Columbia"=>"DC");

In [ ]:
using Plots
using StatsPlots # this package provides stats specific plotting functions
gr()

In [ ]:
using Statistics
using StatsBase
using MLBase

In [ ]:
xtickslabels = ["one","five","six","fourteen"]
p = plot(rand(15),xticks = ([1,5,6,14],xtickslabels),xrotation=90,xtickfont=font(13))

In [ ]:
function pad_empty_plot(p)
    ep = plot(grid=false,legend=false,axis=false,framestyle = :box)#empty plot
    newplot = plot(p,ep,layout=@layout([a{0.99h};b{0.001h}]))
    return newplot
end
pad_empty_plot(p)

In [ ]:
using XLSX
using DataFrames
D = DataFrame(XLSX.readtable("data/zillow_data_download_april2020.xlsx", "Sales_median_price_city")...);
dropmissing!(D)
states = D[:,:StateName];

In [ ]:
NYids = findall(states.=="New York")
NYframe = dropmissing(D[NYids,:])
CAids = findall(states.=="California")
CAframe = dropmissing(D[CAids,:])
FLids = findall(states.=="Florida")
FLframe = dropmissing(D[FLids,:])

Just a quick note about plotting with xlabels that are long and rotated. Currently, there seems to be an issue with using xticks labels that are rotated and long, like the plot I show next. As per this issue https://github.com/JuliaPlots/Plots.jl/issues/2107, this hasn't been fixed yet. But here, I create a quick function that will act as a "hack" to avoid this problem.

One concept I learned from reading one of Edward Tufte's books is the idea of avoiding symmetry. Here, as you can see, each violin plot is symmetric. We can probably fit more information there by making use of each side of the violin plot. And indeed, we will now compare housing prices in these states from February 2020 with housing prices from 10 years before that (February 2010).

### 🔴Plot 1: Symmetric violin plots and annotations
We will get started by just picking the most recent data we have about these states and plot their violin plots to see the distribution of house prices.

In [ ]:
# pick a year: 2020-02
ca = CAframe[!,Symbol("2020-02")]
ny = NYframe[!,Symbol("2020-02")]
fl = FLframe[!,Symbol("2020-02")]

violin(repeat(["New York"], outer=size(ny)), ny,alpha=0.8)
violin!(repeat(["California"], outer=size(ca)), ca,alpha=0.8)
violin!(repeat(["Florida"], outer=size(fl)),fl,alpha=0.8)

In [ ]:
# 2020 data
ca = CAframe[!,Symbol("2020-02")]
ny = NYframe[!,Symbol("2020-02")]
fl = FLframe[!,Symbol("2020-02")]
violin(repeat(["New York"], outer=size(ny)), ny,legend=false,alpha=0.8,side=:right)
violin!(repeat(["California"], outer=size(ca)), ca,alpha=0.8,side=:right)
violin!(repeat(["Florida"], outer=size(fl)),fl,alpha=0.8,side=:right)

### get the February 2010 data
ca10 = CAframe[!,Symbol("2010-02")]
ny10 = NYframe[!,Symbol("2010-02")]
fl10 = FLframe[!,Symbol("2010-02")]

violin(repeat(["New York"], outer=size(ny)), ny10,legend=false,alpha=0.8,side=:left)
violin!(repeat(["California"], outer=size(ca)), ca10,alpha=0.8,side=:left)
violin!(repeat(["Florida"], outer=size(fl)),fl10,alpha=0.8,side=:left)

In [ ]:
# No need for using many colors, let's just use one color for 2010, and one color for 2020

# pick a year: 2019-02
ca = CAframe[!,Symbol("2010-02")]
ny = NYframe[!,Symbol("2010-02")]
fl = FLframe[!,Symbol("2010-02")]
violin(repeat(["New York"], outer=size(ny)), ny,alpha=0.8,side=:left,color=6,label="2010-02")
violin!(repeat(["California"], outer=size(ca)), ca,alpha=0.8,side=:left,color=6,label="")
violin!(repeat(["Florida"], outer=size(fl)),fl,alpha=0.8,side=:left,color=6,label="")

# pick a year: 2020-02
ca = CAframe[!,Symbol("2020-02")]
ny = NYframe[!,Symbol("2020-02")]
fl = FLframe[!,Symbol("2020-02")]
violin(repeat(["New York"], outer=size(ny)), ny,alpha=0.8,side=:right,color=7,label="2020-02")
violin!(repeat(["California"], outer=size(ca)), ca,alpha=0.8,side=:right,color=7,label="")
violin!(repeat(["Florida"], outer=size(fl)),fl,alpha=0.8,side=:right,color=7,label="")

In [ ]:
# pick a year: 2019-02
ca = CAframe[!,Symbol("2010-02")]
ny = NYframe[!,Symbol("2010-02")]
fl = FLframe[!,Symbol("2010-02")]
violin(repeat(["New York"], outer=size(ny)), ny,alpha=0.8,side=:left,color=6,label="2010-02")
violin!(repeat(["California"], outer=size(ca)), ca,alpha=0.8,side=:left,color=6,label="")
violin!(repeat(["Florida"], outer=size(fl)),fl,alpha=0.8,side=:left,color=6,label="")

# pick a year: 2020-02
ca = CAframe[!,Symbol("2020-02")]
ny = NYframe[!,Symbol("2020-02")]
fl = FLframe[!,Symbol("2020-02")]
violin(repeat(["New York"], outer=size(ny)), ny,alpha=0.8,side=:right,color=7,label="2020-02")
violin!(repeat(["California"], outer=size(ca)), ca,alpha=0.8,side=:right,color=7,label="")
violin!(repeat(["Florida"], outer=size(fl)),fl,alpha=0.8,side=:right,color=7,label="")


m = median(ny)
ep = 0.1
annotate!([(0.5+ep,m+0.05,text(m/1000,10,:left))])

m = median(ca)
ep = 0.1
annotate!([(1.5+ep,m+0.05,text(m/1000,10,:left))])

m = median(fl)
ep = 0.1
annotate!([(2.5+ep,m+0.05,text(m/1000,10,:left))])

plot!(xtickfont=font(10),size=(500,300))

In [ ]:
# putting it together.

ep = 0.05 # will later be used in padding for annotations

# set up the plot
plot(xtickfont=font(10))

states_of_interest = ["New York", "California", "Florida", "Ohio","Idaho"]
years_of_interst = [Symbol("2010-02"),Symbol("2020-02")]

# year 1
xstart = 0.5
yi = years_of_interst[1]
for si in states_of_interest
    curids = findall(states.==si)
    curFrame = D[curids,:]
    curprices = curFrame[!,yi]
    m = median(curprices)
    annotate!([(xstart-ep,m+0.05,text(m/1000,8,:right))])
    xstart += 1
    violin!(repeat([si], outer=size(curprices)), curprices,alpha=0.8,side=:left,color=6,label="")
end
plot!(Shape([],[]),color=6,label=yi)

# year 2
xstart = 0.5
yi = years_of_interst[2]
for si in states_of_interest
    curids = findall(states.==si)
    curFrame = D[curids,:]
    curprices = curFrame[!,yi]
    m = median(curprices)
    annotate!([(xstart+ep,m+0.05,text(m/1000,8,:left))])
    xstart += 1
    violin!(repeat([si], outer=size(curprices)), curprices,alpha=0.8,side=:right,color=7,label="")
end
plot!(Shape([],[]),color=7,label=yi)
ylabel!("housing prices")

### 🔴Plot 2: Bar charts, histograms, and insets
Now let's compare states based on the number of location entries they have in the data.

In [ ]:
mapstates = labelmap(states)
stateids = labelencode(mapstates, states)
histogram(stateids,nbins=length(mapstates))

There are a few problems with this histogram. First, unsorted histograms are often harder to read so the first thing we will do is rearrange this histogram. Next, we will add annotations to be able to map each bar to a state quickly.

In [ ]:
# first we'll start with sorting
h = fit(Histogram, stateids,nbins=length(mapstates))
sortedids = sortperm(h.weights,rev=true)
bar(h.weights[sortedids],legend=false)

In [ ]:
bar(h.weights[sortedids],legend=false,orientation = :horizontal,yflip=true)

In [ ]:
# just an example of annotations
bar(h.weights[sortedids],legend=false,orientation = :horizontal,yflip=true,size=(400,500))
stateannotations = mapstates.vs[sortedids]
for i = 1:3
    annotate!([(h.weights[sortedids][i]-5,i,text(stateannotations[i],10,:left))])
end
plot!()

In [ ]:
bar(h.weights[sortedids],legend=false,orientation = :horizontal,yflip=true,linewidth=0,width=0,size=(400,500))
stateannotations = mapstates.vs[sortedids]
for i = 1:length(stateannotations)
    annotate!([(h.weights[sortedids][i]-5,i,text(stateabbreviations[stateannotations[i]],5,:left))])
end
plot!()

In [ ]:
bar(h.weights[sortedids],legend=false,orientation = :horizontal,
        yflip=true,linewidth=0,width=0,color=:gray,alpha=0.8)
stateannotations = mapstates.vs[sortedids]
for i = 20:20:200
    plot!([i,i],[50,0],color=:white)
end
for i = 1:length(stateannotations)
    annotate!([(h.weights[sortedids][i]-5,i,text(stateabbreviations[stateannotations[i]],6,:left))])
end
plot!(grid=false,yaxis=false,xlim=(0,maximum(h.weights)),xticks = 0:20:200)
xlabel!("number of listings")

In [ ]:
bar(h.weights[sortedids],legend=false,orientation = :horizontal,
        yflip=true,linewidth=0,color=:gray,alpha=0.8,size=(300,500))
stateannotations = mapstates.vs[sortedids]
ht = length(h.weights)
for i = 20:20:200
    plot!([i,i],[ht,0],color=:white)
end
for i = 1:length(stateannotations)
    annotate!([(h.weights[sortedids][i]+2,i,text(stateabbreviations[stateannotations[i]],6,:left))])
end
plot!(grid=false,yaxis=false,xlim=(0,maximum(h.weights)+5),xticks = 0:20:200)
xlabel!("number of listings")

f = Plots.plot!(inset = bbox(0.7,0.15,0.25,0.6,:top,:left))
bar!(f[2],h.weights[sortedids][21:end],legend=false,orientation = :horizontal,
        yflip=true,linewidth=0,width=0,color=:gray,alpha=0.8)
for i = 21:length(stateannotations)
    annotate!(f[2],[(h.weights[sortedids][i]+1,i-20,text(stateabbreviations[stateannotations[i]],6,:left))])
end
plot!(f[2],[10,10],[20,0],color=:white,xticks=0:10:20,yaxis=false,grid=false,xlim=(0,20))
plot!()

### 🔴Plot 3: Plots with error bars
Next, we will compar state prices over the years and see how they have changed. we will use error bars too.

In [ ]:
M = Matrix(NYframe[:,5:end])

In [ ]:
xtickslabels = string.(names(NYframe[!,5:end]))

In [ ]:
plot()
for i = 1:size(M,1)
    plot!(M[i,:],legend=false)
end
plot!()
p = plot!(xticks = (1:4:length(xtickslabels),xtickslabels[1:4:end]),xrotation=90,xtickfont=font(8),grid=false)
pad_empty_plot(p)

A plot like this isn't indicative of how the price trend is going overall for New York. What we will do next is for each time point, we will find the median value as well as the 80th and 20th percentile and plot these values. Let's write the precentile functin first.

In [ ]:
function find_percentile(M, pct)
    r = zeros(size(M,2))
    for i = 1:size(M,2)
        v = M[:,i]
        len = length(v)
        ind = floor(Int64,pct*len)
        newarr = sort(v);
        r[i] = newarr[ind];
    end
    return r
end

md = find_percentile(M,0.5)
mx = find_percentile(M,0.8)
mn = find_percentile(M,0.2)
plot(md,ribbon =(md.-mn,mx.-md),color = :blue,label="NY",grid=false)
p = plot!(xticks = (1:4:length(xtickslabels),xtickslabels[1:4:end]),xrotation=90,xtickfont=font(8))
pad_empty_plot(p)

In [ ]:
function plot_individual_state!(plotid,statevalue,colorid)
    curids = findall(states.==statevalue)
    curFrame = D[curids,:]
    M = Matrix(curFrame[:,5:end])
    md = find_percentile(M,0.5)
    mx = find_percentile(M,0.8)
    mn = find_percentile(M,0.2)
    plot!(plotid,md,ribbon =(md.-mn,mx.-md),color = colorid,label=stateabbreviations[statevalue],grid=false)
    plot!(plotid,xticks = (1:4:length(xtickslabels),xtickslabels[1:4:end]),xrotation=90,xtickfont=font(8))
end

In [ ]:
plotid = plot()
plot_individual_state!(plotid,"Indiana",1)
plot_individual_state!(plotid,"Ohio",2)
plot_individual_state!(plotid,"Idaho",3)
# plot_individual_state!(plotid,"California",4)
ylabel!("prices")
pad_empty_plot(plotid)

### 🔴Plot 4: Plots with double axes

In [ ]:
vector1 = rand(10)
vector2 = rand(10)*100
plot(vector1,label = "b",size=(300,200))
plot!(twinx(), vector2,color=2,axis=false)

In [ ]:
xtickslabels = NYframe[!,:RegionName]

In [ ]:
sz = NYframe[!,:SizeRank]
pc = NYframe[!,end]
M = Matrix(NYframe[:,5:end])
M = copy(M')
md = find_percentile(M,0.9)

md = find_percentile(M,0.5)
mx = find_percentile(M,0.9)
mn = find_percentile(M,0.1)
vector1 = sz

plot()
plot!(md,ribbon =(md.-mn,mx.-md),color = 1,grid=false,label="")

plot!(xticks = (1:length(xtickslabels),xtickslabels),xrotation=90,xtickfont=font(10))
plot!(twinx(), vector1,color=2,label="",ylabel="rank",grid=false,xticks=[],linewidth=2)
plot!(Shape([],[]),color=1,label="Prices (left)")
p = plot!([],[],color=2,label="Rank (right)")
ep = plot(grid=false,legend=false,axis=false,framestyle = :box)#empty plot
plot(p,ep,layout=@layout([a{0.85h};b{0.001h}]))

### 🔴Plot 5: High-dimensional data in a 2D plot
We've seen a 3D plot in the Clustering notebook previously. I personally prefer 2D plots because they are often easier to read and is viewable in a print version more clearly. Here, we will explore how we can use color as a third dimension. Note that you can also use sizes as third dimension.

We will use the California data, and plot the prices from 2010-02 on the x-axis and 2020-02 on the y-axis. We will then color code each data point by its current rank.

Let's generate a quick scatter plot first.

In [ ]:
CA202002 = CAframe[!,Symbol("2020-02")]
CA201002 = CAframe[!,Symbol("2010-02")]
scatter(CA201002,CA202002)

In [ ]:
CA202002 = CAframe[!,Symbol("2020-02")]
CA201002 = CAframe[!,Symbol("2010-02")]
CAranks = CAframe[!,:SizeRank]
scatter(CA201002,CA202002,legend=false,markerstrokewidth=0,markersize=3,alpha=0.6,grid=false)

In [ ]:
import Pkg
Pkg.add("ColorSchemes")
using ColorSchemes

In [ ]:
# normalize the ranks to be between 0 and 1
continuousranks = CAranks./maximum(CAranks)

# create a placeholder vector that will store the color of each value
colorsvec = Vector{RGB{Float64}}(undef,length(continuousranks))

# and finally map the colors according to ColorSchemes.autumn1, there are many other schemes you can choose from
map(i->colorsvec[i]=get(ColorSchemes.autumn1,continuousranks[i]),1:length(colorsvec))

In [ ]:
continuousdates = CAranks./maximum(CAranks)
colorsvec = Vector{RGB{Float64}}(undef,length(continuousdates))
map(i->colorsvec[i]=get(ColorSchemes.autumn1,continuousdates[i]),1:length(colorsvec))
scatter(CA201002,CA202002,color=colorsvec,
    legend=false,markerstrokewidth=0,markersize=3,grid=false)
xlabel!("2010-02 prices",xguidefontsize=10)
ylabel!("2020-02 prices",yguidefontsize=10)
p1 = plot!()

In [ ]:
#set up the plot canvas
xvals = 0:100
s = Shape([0,1,1,0],[0,0,1,1])
plot(s,color=ColorSchemes.autumn1[1],grid=false,axis=false,
    legend=false,linewidth=0,linecolor=nothing)

for i = 2:101
    s = Shape([xvals[i],xvals[i]+1,xvals[i]+1,xvals[i]],[0,0,1,1])
    plot!(s,color=ColorSchemes.autumn1[i],grid=false,axis=false,
    legend=false,linewidth=0,linecolor=nothing)
end

mynormalizer = maximum(CAranks)
xtickslabels = 0:div(mynormalizer,10):mynormalizer
continuousdates = xtickslabels./mynormalizer
xticksloc = round.(Int,continuousdates.*101)

# annotate using the ranks
rotatedfont = font(10, "Helvetica",rotation=90)
for i = 1:length(xtickslabels)
    annotate!(xticksloc[i],0.5,text(xtickslabels[i], rotatedfont))
end
p2 = plot!()

In [ ]:
mylayout = @layout([a{0.89h};b{0.1h}])
plot(p1,p2,layout=mylayout)

# Finally...
After finishing this notebook, you should be able to:
- [ ] create violin plots in julia
- [ ] create bar charts
- [ ] add annotations to your plots
- [ ] create an inset figure for your plot
- [ ] create plots with error margin
- [ ] create plots with double axes
- [ ] create a new color mapping to a given set of values
- [ ] create two dimensional plots and use color to indicate a third dimension
- [ ] pad multiple plots together

# 🥳 One cool finding

Many interesting cool things here! The most interesting I found was that Idaho is following California's trend in housing prices, and Idaho's prices are growing faster than places like Indiana and Ohio.

<img src="data/1201.png" width="500">
<img src="data/1202.png" width="500">